In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [3]:
df=pd.read_csv('diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
x=df.drop(columns=['Outcome']).values
y=df['Outcome'].values

In [7]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2)


In [8]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [9]:
# scaling the data 

In [10]:
from sklearn.preprocessing import  StandardScaler
scale=StandardScaler()
xtrain=scale.fit_transform(xtrain)
xtest=scale.fit_transform(xtest)

# Model without keras tuner

In [11]:
temp_model=Sequential()
temp_model.add(Dense(256,activation='relu',input_dim=8))
temp_model.add(Dense(128,activation='relu'))
temp_model.add(Dense(1,activation='sigmoid'))
temp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2304      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 35329 (138.00 KB)
Trainable params: 35329 (138.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
temp_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
temp_model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50)

Epoch 1/50
20/20 [==============================] - 2s 18ms/step - loss: 0.5562 - accuracy: 0.7280 - val_loss: 0.4992 - val_accuracy: 0.7857
Epoch 2/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4670 - accuracy: 0.7720 - val_loss: 0.4802 - val_accuracy: 0.7662
Epoch 3/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4447 - accuracy: 0.7818 - val_loss: 0.4854 - val_accuracy: 0.7662
Epoch 4/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4372 - accuracy: 0.7801 - val_loss: 0.4918 - val_accuracy: 0.7662
Epoch 5/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4280 - accuracy: 0.7932 - val_loss: 0.4933 - val_accuracy: 0.7468
Epoch 6/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4266 - accuracy: 0.7899 - val_loss: 0.4961 - val_accuracy: 0.7532
Epoch 7/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4209 - accuracy: 0.7948 - val_loss: 0.4985 - val_accuracy: 0.7338
Epoch 8/50
20/20 [=

So problems in this model is we dont able to know:

- how many layers we have to add

- how many node a particular layer have

- which loss function is best for each layer

- which activation function is suitable for every layer

- how many epoch the model have to train 

Here the keras tuner is came into the picture

# Model with keras tuner

In [14]:
import kerastuner as kt

C:\Users\Asus\AppData\Local\Temp\ipykernel_19736\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


There are 6 steps 

- first is you have to make a function named "build_model(hp)" // where hp is the object(hyperparameter)

- second is you have to make "tuner object" of a class RandomSearch()

- you have to call the "search()" function using tuner object

- extract best hyperparameter 

- Get Best model

- make the model using the best hyperparameter we got

## how to tune the appropriate optimizer

- step1: build_model()

In [18]:
def build_model(hp):
    """build a model"""
    model=Sequential()
    model.add(Dense(256,activation='relu',input_dim=8))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']) # a function which takes two thing 1st is string and 2nd is list
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

- step2: make a tuner object of class RandomSearch

In [21]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5) 

Reloading Tuner from .\untitled_project\tuner0.json


- step3: call search() using tuner object

In [22]:
tuner.search(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.3636363744735718

Best val_accuracy So Far: 0.6753246784210205
Total elapsed time: 00h 08m 42s


- step4: get best hyperparameter

In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

- step5: Get best model 

In [25]:
model=tuner.get_best_models(num_models=1)[0]

- step6: train the model

In [27]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),initial_epoch=6,epochs=100) # initial_epoch is 6 because we already run 5 epochs

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.5994 - accuracy: 0.7036 - val_loss: 0.6018 - val_accuracy: 0.6753
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5861 - accuracy: 0.7231 - val_loss: 0.5909 - val_accuracy: 0.6883
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5740 - accuracy: 0.7264 - val_loss: 0.5810 - val_accuracy: 0.6948
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5634 - accuracy: 0.7296 - val_loss: 0.5718 - val_accuracy: 0.7273
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5530 - accuracy: 0.7492 - val_loss: 0.5634 - val_accuracy: 0.7532
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5435 - accuracy: 0.7557 - val_loss: 0.5562 - val_accuracy: 0.7597
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5355 - accuracy: 0.7557 - val_loss: 0.5488 - val_accuracy: 0.7532
Epoch 14